In [1]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.preprocessing import LabelEncoder

In [2]:
images=[]
globalpath='C:\\Users\\jerar\\OneDrive\\Desktop\\food_classify\\indian_foods'
os.chdir(globalpath)
print(os.getcwd())
labels=os.listdir()#list of all foodnames since food name is the directory name
print(labels)
class_len=len(labels)
print(class_len)

C:\Users\jerar\OneDrive\Desktop\food_classify\indian_foods
['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala', 'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua', 'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma', 'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha', 'poornalu', 'po

In [3]:
data={"image":[],
    "label":[]
}
for label in labels:
    os.chdir(globalpath)
    path=os.getcwd()+"\\"+label
    os.chdir(path)
    imagelist=os.listdir()
    labellist=[]
    for i in range(0,len(imagelist)):
        imagelist[i]=path+"\\"+imagelist[i]
        labellist.append(label)
    data["image"].extend(imagelist)
    data["label"].extend(labellist)


'''
data["label"]=list(encoded_labels)'''
#load data into a DataFrame object:
df = pd.DataFrame(data)
df = df.sample(frac=1).reset_index(drop=True) # shuffling dataframe
print(df.head())



                                               image             label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...         adhirasam
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...        aloo_tikki
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...             lassi
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...        sutar_feni
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...  double_ka_meetha


In [4]:
#encoding labels
encoder = LabelEncoder()
encoded_labels= encoder.fit_transform(df.iloc[:,1])
df.iloc[:,1]=encoded_labels
df['label']=df['label'].astype(str)
print(df.head())


                                               image label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...     5
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...    47
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...    78
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...    30


In [5]:
### creating tensors from dictionary using tf.data.Dataset.from_tensor_slices(tensors)
### tensors should be 1D arrays
###

In [6]:
train_df,test_df=train_test_split(df,test_size = 0.30, shuffle = True, random_state = 1)

In [7]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,validation_split = 0.2) # rescale + augmentation
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255) # only rescale

In [8]:
train_image = train_gen.flow_from_dataframe(dataframe = train_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 32,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = True,
                                           seed = 42,
                                           subset = 'training')
val_image = train_gen.flow_from_dataframe(dataframe = train_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 32,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = True,
                                           seed = 42,
                                           subset = 'validation')
test_image = test_gen.flow_from_dataframe(dataframe = test_df,
                                           x_col = 'image',
                                           y_col = 'label',
                                           target_size = (224,224),
                                           batch_size = 32,
                                           color_mode = 'rgb',
                                           class_mode = 'categorical',
                                           shuffle = False)

Found 2240 validated image filenames belonging to 80 classes.


C:\Users\jerar\anaconda3\envs\foodclassify\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 559 validated image filenames belonging to 80 classes.


C:\Users\jerar\anaconda3\envs\foodclassify\lib\site-packages\keras\preprocessing\image.py:1139: UserWarning: Found 1 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  warnings.warn(


Found 1201 validated image filenames belonging to 80 classes.


In [9]:
inputs = tf.keras.Input(shape = (120,120,3))
x = tf.keras.layers.Conv2D(filters = 16, kernel_size =(3,3), activation = 'relu' )(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size =(3,3), activation = 'relu' )(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation = 'relu')(x) #hidden layer
x = tf.keras.layers.Dense(64, activation = 'relu')(x)
outputs = tf.keras.layers.Dense(1, activation = 'linear')(x) #single neuron with linear activation function
model = tf.keras.Model(inputs = inputs, outputs = outputs)
model.compile(optimizer = 'adam',
             loss = 'mse',
             metrics = ['accuracy'])
history  = model.fit(train_image,
                    validation_data = val_image,
                    epochs = 100,
                    callbacks = [tf.keras.callbacks.EarlyStopping(
                    monitor = 'val_loss',
                    patience = 5,
                    restore_best_weights = True)])

Epoch 1/100
70/70 [==============================] - 59s 836ms/step - loss: 0.0124 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 2/100
70/70 [==============================] - 31s 447ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 3/100
70/70 [==============================] - 31s 441ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 4/100
70/70 [==============================] - 31s 446ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 5/100
70/70 [==============================] - 36s 521ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 6/100
70/70 [==============================] - 51s 728ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy: 0.9875
Epoch 7/100
70/70 [==============================] - 51s 724ms/step - loss: 0.0123 - accuracy: 0.9875 - val_loss: 0.0123 - val_accuracy:

In [10]:
result = model.evaluate(test_image, verbose = 0)
print('Test Loss : {:.3f}'.format(result[0]))
print('Test Accuracy : {:.3f}%'.format(result[1]*100))

Test Loss : 0.012
Test Accuracy : 98.750%


In [11]:
model.save("model")

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [12]:
import os
os.getcwd()

'C:\\Users\\jerar\\OneDrive\\Desktop\\food_classify\\indian_foods\\unni_appam'

In [27]:
converter = tf.lite.TFLiteConverter.from_saved_model("model")
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)